In [47]:
# 데이터 준비
import pandas as pd

campaign_df = pd.read_csv('campaign_master.csv')
class_df = pd.read_csv('class_master.csv')
customer_df = pd.read_csv('customer_master.csv')
use_df = pd.read_csv('use_log.csv')

In [48]:
# 병합
custonmer_join = pd.merge(customer_df, use_df, on='customer_id', how='outer')
custonmer_join = pd.merge(custonmer_join, class_df, on='class', how='left')
custonmer_join = pd.merge(custonmer_join, campaign_df, on='campaign_id', how='left')
custonmer_join.columns.unique()

Index(['customer_id', 'name', 'class', 'gender', 'start_date', 'end_date',
       'campaign_id', 'is_deleted', 'log_id', 'usedate', 'class_name', 'price',
       'campaign_name'],
      dtype='object')

In [49]:
# 결측치 처리
# custonmer_join.isnull().sum()
custonmer_join = custonmer_join.dropna()

custonmer_join.rename(columns={'end_date': 'calc_date'}, inplace=True)
custonmer_join.isnull().sum()

customer_id      0
name             0
class            0
gender           0
start_date       0
calc_date        0
campaign_id      0
is_deleted       0
log_id           0
usedate          0
class_name       0
price            0
campaign_name    0
dtype: int64

In [50]:
# 멤버십 기간 계산(calc_date - start_date)
print(custonmer_join.shape)
custonmer_join['calc_date'] = pd.to_datetime(custonmer_join['calc_date'])
custonmer_join['start_date'] = pd.to_datetime(custonmer_join['start_date'])

custonmer_join['date_difference'] = (custonmer_join['calc_date'] - custonmer_join['start_date']).dt.days
custonmer_join['date_difference'].shape

(29474, 13)


(29474,)

In [66]:
# use_log 데이터를 활용하여 월별 이용횟수(평균, 최대, 최소, 중앙값) 계산
use_df['usedate'] = pd.to_datetime(use_df['usedate'])
use_df['month'] = use_df['usedate'].dt.to_period('M')

# grouped = use_df.groupby('month').size()
# print(grouped)

# month_counts = grouped.reset_index(name='count')
# month_counts['count'].agg(['mean', 'max', 'min', 'median']).reset_index()

from sklearn.preprocessing import LabelEncoder

# object → int
le_disaster = LabelEncoder()
use_df['customer_id'] = le_disaster.fit_transform(use_df['customer_id'])

groupby_agg = use_df.groupby('customer_id')['month'].agg(['mean', 'median', 'max', 'min'])
groupby_agg

,mean,median,max,min
customer_id,,,,
0,2018-09,2018-09,2019-03,2018-04
1,2018-09,2018-08,2019-03,2018-06
2,2018-04,2018-04,2018-04,2018-04
3,2018-09,2018-10,2019-03,2018-04
4,2018-09,2018-10,2019-03,2018-05
...,...,...,...,...
4187,2019-02,2019-02,2019-03,2019-02
4188,2018-11,2018-12,2019-03,2018-09
4189,2018-09,2018-10,2019-03,2018-04
